In [ ]:
import datetime as dt
import pandas as pd
import redis

from fastcounting import helper

In [ ]:
r = redis.Redis(**helper.Helper().rediscred, decode_responses=True)

In [ ]:
lua_total = """
local result = 0.0
for l, atomic in ipairs(redis.call('SMEMBERS', KEYS[1]))
do result = result + redis.call('HGET', 'atomicID:' .. atomic, 'amount')
end
return result
"""

In [ ]:
def read_lexware(month, name='report', nrows=None):
    """Read xlxs from the default folder for each year e.g. month=2018-13 or actual month."""
    p = helper.Helper().datafolder(month)
    files = [file for file in p.iterdir() if file.parts[-1].lower().startswith(name)]
    if files:
        data = pd.read_table(files[0], sep='\t', engine='python', header=[0, 1], decimal=',', thousands='.', parse_dates=[2])
    return data

In [ ]:
def clean_report(validate):
    better_columns = ['Konto', 'Name', 'Letzte Buchung', 'EB Soll', 'EB Haben', 'Summe Soll', 'Summe Haben', 'drop1', 'drop2', 'Saldo Soll', 'Saldo Haben']
    validate.columns = better_columns
    validate.set_index('Konto', inplace = True)
    validate.fillna(0, inplace=True)  # for full year data set they are equal to sum
    validate.drop(['drop1', 'drop2'], axis=1, inplace=True)
    return validate

In [ ]:
def total_diff(validate, lua_script):
    """Check total sum which is aggregation of EB + Saldo Haben + Saldo Soll."""
    redis_sum_list = []
    for konto in validate.index:
        redis_sum_list.append(
            r.eval(lua_script, 1, f'account:atomic:{float(konto)}'))
    validate['test_redis'] = redis_sum_list
    validate['test_redis'] = validate['test_redis'] / 100
            
    validate['checksum'] =  validate['Saldo Haben'] - validate['Saldo Soll']
    return validate[validate['checksum'] != validate['test_redis']]

In [ ]:
# main
validate = read_lexware('2017-13')
validate = clean_report(validate)

In [ ]:
total_diff(validate, lua_total)

In [ ]:
# date thingy

In [ ]:
atomics = r.zrangebyscore('atomic:date', dt.datetime(2017,2,1).timestamp(), dt.datetime(2017,2,2).timestamp(), withscores=True)

In [ ]:
result_list = []
for atomic in atomics:
    row = r.hgetall(f'atomicID:{atomic[0]}')
    row.update({'Buchungsdatum':dt.datetime.fromtimestamp(atomic[1])})
    result_list.append(row)

In [ ]:
dateview = pd.DataFrame(result_list)

In [ ]:
r.hgetall('batchID:1')